In [6]:
from autoadsorbate import Surface
from ase.build import fcc211
from ase.constraints import FixAtoms
from ase.visualize import view
from ase import Atom, Atoms
import numpy as np

from autoadsorbate.Surf import _get_starting_grid, get_shrinkwrap_grid, shrinkwrap_surface
from autoadsorbate.utils import get_blenderized

In [7]:
def interpolate_neb_trajectory(atoms_a, atoms_b, image_no = 5):
    """helper funct to quickly make linear interpolations"""
    from ase.mep import interpolate
    
    if len(atoms_a) != len(atoms_b): # Safety checks
        raise ValueError("Atoms objects must have the same number of atoms.")
    if atoms_a.get_chemical_symbols() != atoms_b.get_chemical_symbols():
        raise ValueError("Atoms must have the same atomic symbols in the same order.")
    
    images = [atoms_a.copy()]
    for _ in range(image_no):
        images.append(atoms_a.copy())
    images.append(atoms_b.copy())
    interpolate(images)
    return images

def frames_from_info(info):
    out = interpolate_neb_trajectory(
        atoms_a=info['traj'][0],
        atoms_b=info['traj'][-1],
        image_no = info['frames_repeat']
        )
    return out

## make an "easy to read" but interesting slab

In [8]:
prec = 0.3
tss = 2.2

slab = fcc211('Cu', size=[6,3,4], vacuum=10)
slab.positions[:,0]+=2
slab.wrap()

# get surface, sub surface and sub sub surface
# maybe useful method for the Surface class?
motv = [0,0,10] #move_out_of_the_way_vector
surf_inds = shrinkwrap_surface(slab)
slab.positions-= [motv if atom.index in surf_inds else [0,0,0]  for atom in slab]
sub_surf_inds = shrinkwrap_surface(slab)
slab.positions-= [motv if atom.index in sub_surf_inds else [0,0,0]  for atom in slab]
sub_sub_surf_inds = shrinkwrap_surface(slab)

slab.positions+=[motv if atom.index in list(surf_inds)+list(sub_surf_inds) else [0,0,0]  for atom in slab]

unfrozen = list(surf_inds)+list(sub_surf_inds)+list(sub_sub_surf_inds)

for i in unfrozen:
    view_slab = slab.copy()
    view_slab[i].symbol ='Zn'
# view(slab)

slab.set_constraint(FixAtoms(indices=[atom.index for atom in slab if atom.index not in unfrozen]))
slab.rattle(stdev=0.2)

In [5]:
view(slab)

<Popen: returncode: None args: ['/home/djrm/venv/mace_env/bin/python', '-m',...>

## Init Surface object

In [9]:
s = Surface(slab, precision=prec, touch_sphere_size=tss)
marked_surface = s.view_surface(return_atoms=True)

Visualizing surface Cu atoms as Zn


## prep assets for render

In [10]:
starting_grid = _get_starting_grid(slab, precision=prec)

high_grid = starting_grid.copy()
high_grid.positions[:,2] += (high_grid.positions[:,0]+4)**1.1 #high_grid.positions[:,0] + 10
high_grid.positions[:,2] += (high_grid.positions[:,1]+3)**2*0.1 #high_grid.positions[:,0] 


grid = get_shrinkwrap_grid(slab, precision=prec, touch_sphere_size=tss)


In [23]:
# view([s.view_site(i, return_atoms=True) for i in s.site_df.index.values])
site = s.view_site(39, return_atoms=True)
site_vector = site[[atom.index for atom in site if atom.symbol != 'Fe']]
for atom in site_vector:
    atom.symbol = 'H'
    
site = site[[atom.index for atom in site if atom.symbol == 'Cu']]
for atom in site:
    if atom.symbol=='Cu':
        atom.symbol='Zn'

hedgehog=slab.copy()[[]]
for a in [s.view_site(i, return_atoms=True) for i in s.site_df.index.values]:
    hedgehog+=a
    
hedgehog=hedgehog[[atom.index for atom in hedgehog if atom.symbol == 'X']]
for atom in hedgehog:
    atom.symbol='H'
            
site_marker = Atoms(['Pt'], [site.info['coordinates']])
site_sphere = Atoms(['Cd'], [site.info['coordinates']])

site_blanket = grid[[atom.index for atom in grid if np.linalg.norm(atom.position-site.info['coordinates']) < tss *1.2]]
for atom in site_blanket:
    atom.symbol='H'
    
nvector_marker = site_blanket.copy()
nvector_marker.positions = np.array([site_blanket.get_center_of_mass()])

grid_marker = grid[[int(np.argmin(np.linalg.norm(grid.get_positions() - site_vector[-1].position, axis=1)))]]
grid_marker[0].symbol = 'Br'

grid_sphere = grid_marker.copy()
grid_sphere[0].symbol = 'F'

In [24]:
# view([site+hedgehog+site_marker+site_blanket+nvector_marker])
# view([site+grid_marker])

## directing the movie

In [25]:
rslab = marked_surface #+ slab

fps = 24
def stofps(s):
    return max([1, int(-2+fps*s)])

composition = [
    {'traj': [
        slab+high_grid,
        slab+starting_grid
        ], 'frames_repeat':100},               # drop grid
    {'traj': [
        slab+starting_grid,
        slab+grid
        ], 'frames_repeat':100},               #shrinkwrap         
    {'traj': [
        marked_surface+grid+grid_marker,
        marked_surface+grid+grid_marker
              ], 'frames_repeat':50},       
    {'traj': [
        slab+grid+grid_marker+grid_sphere+site,
        slab+grid+grid_marker+grid_sphere+site
        ], 'frames_repeat':50},
    {'traj': [
        slab+grid+site+site_marker,
        slab+grid+site+site_marker
        ], 'frames_repeat':50},
    {'traj': [
        slab+grid+site+site_marker+site_sphere+site_blanket,
        slab+grid+site+site_marker+site_sphere+nvector_marker
        ], 'frames_repeat':100}, 
    {'traj': [
        slab+site+site_vector,
        slab+site+site_vector
        ], 'frames_repeat':50},
    {'traj': [
        marked_surface+hedgehog,
        marked_surface+hedgehog
        ], 'frames_repeat':100}
    
]

render_traj = []

for info in composition:
    render_traj += frames_from_info(info)

In [ ]:
from ase.io import write
render_traj = get_blenderized(render_traj, hide_spot=[0,0,0])
write('./render_traj.xyz', render_traj)

In [22]:
view(render_traj)
# view(high_grid)

<Popen: returncode: None args: ['/home/djrm/venv/mace_env/bin/python', '-m',...>